<a href="https://colab.research.google.com/github/goransavich/cnn-xgboost/blob/main/cnn_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot
np.random.seed(1337)  # for reproducibility
#from keras.utils import np_utils
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Flatten, Input, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import matplotlib.pyplot as plt
#xgboost
import xgboost as xgb
import pandas as pd
from sklearn import metrics

In [22]:
from google.colab import files
uploaded = files.upload()
import io
import pandas as pd

train_name = list(uploaded.keys())[0]

train_df = pd.read_csv(io.BytesIO(uploaded[train_name]))

Saving Beef_TRAIN.csv to Beef_TRAIN.csv


In [23]:
uploaded = files.upload()

test_name = list(uploaded.keys())[0]

test_df = pd.read_csv(io.BytesIO(uploaded[test_name]))

Saving Beef_TEST.csv to Beef_TEST.csv


In [24]:
train_data = np.array(train_df)
test_data = np.array(test_df)

In [25]:
x_train = train_data[:, :-1]
x_test = test_data[:, :-1]
train_y = train_data[:, -1:] ###OVDE VIDETI KAKO DA NAPRAVIM Y TEST DA IMA 22 KOMADA
test_y = test_data[:, -1:]

classes_in_dataset = len(np.unique(np.concatenate((train_y, test_y), axis=0)))

# transform the labels from integers to one hot vectors
enc = OneHotEncoder(categories='auto')
enc.fit(np.concatenate((train_y, test_y), axis=0).reshape(-1, 1))
y_train = enc.transform(train_y.reshape(-1, 1)).toarray()
y_test = enc.transform(test_y.reshape(-1, 1)).toarray()


In [26]:
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 42)

In [27]:
if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
input_shape = x_train.shape[1:]

print(input_shape)

print(classes_in_dataset)
###y_train for xgboost without encoding
y_train_xgboost = enc.inverse_transform(y_train)


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validate.shape)
print(y_validate.shape)

(470, 1)
5
(24, 470, 1)
(24, 5)
(30, 470, 1)
(30, 5)
(6, 470)
(6, 5)


In [28]:
inputs = input_shape
input_layer = Input(inputs)
conv2d1 = Convolution1D(filters=64, kernel_size=1, padding='valid')(input_layer)
activation1 = Activation('relu')(conv2d1)
maxpooling1 = MaxPooling1D(pool_size=2, strides=1, padding='valid')(activation1)
#dropout1 = Dropout(0.5)(maxpooling1)

conv2d2 = Convolution1D(filters=62, kernel_size=1, padding='valid')(maxpooling1)
activation2 = Activation('relu')(conv2d2)
maxpooling2 = MaxPooling1D(pool_size=2, strides=1, padding='valid')(activation2)
dropout2 = Dropout(0.5)(maxpooling2)

flatten = Flatten()(maxpooling2)

dense1 = Dense(256)(flatten)
activation3 = Activation('relu')(dense1)

dense2 = Dense(units=classes_in_dataset)(activation3)
activation4 = Activation('softmax')(dense2)

output_for_xgboost = Model(inputs=input_layer, outputs=dense1)
model = Model(inputs=input_layer, outputs=activation4)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.compile(loss ='categorical_crossentropy', optimizer=Adam(learning_rate=0.001),metrics =['accuracy'])

history = model.fit(
    x_train,
    y_train,
    batch_size=2,
    epochs=75,
    verbose=1,
    callbacks=[callback],
    validation_data=(x_validate,y_validate),
)

score = model.evaluate(x_test,y_test,verbose=0)
print('Test Loss : {:.4f}'.format(score[0]))
print('Test Accuracy : {:.4f}'.format(score[1]))


Epoch 1/75
12/12 [==============================] - 2s 112ms/step - loss: 2.3333 - accuracy: 0.2083 - val_loss: 1.5844 - val_accuracy: 0.3333
Epoch 2/75
12/12 [==============================] - 1s 97ms/step - loss: 1.2714 - accuracy: 0.3750 - val_loss: 1.6459 - val_accuracy: 0.1667
Epoch 3/75
12/12 [==============================] - 1s 94ms/step - loss: 1.1107 - accuracy: 0.5000 - val_loss: 1.8400 - val_accuracy: 0.1667
Epoch 4/75
12/12 [==============================] - 1s 94ms/step - loss: 1.0194 - accuracy: 0.6250 - val_loss: 1.9770 - val_accuracy: 0.1667
Epoch 5/75
12/12 [==============================] - 1s 96ms/step - loss: 0.8639 - accuracy: 0.6250 - val_loss: 2.0575 - val_accuracy: 0.3333
Epoch 6/75
12/12 [==============================] - 1s 97ms/step - loss: 0.7507 - accuracy: 0.7083 - val_loss: 1.9325 - val_accuracy: 0.1667
Epoch 7/75
12/12 [==============================] - 1s 93ms/step - loss: 0.8295 - accuracy: 0.6667 - val_loss: 1.8491 - val_accuracy: 0.5000
Epoch 8/75
1

In [29]:
######### XGBOOST #############

intermediate_layer_model = output_for_xgboost

intermediate_layer_model.summary()

x_train_xgboost = intermediate_layer_model.predict(x_train)
x_test_xgboost = intermediate_layer_model.predict(x_test)

#label encoder for y to be in range from 0 to num-classes
def transform_labels(y_train_xgboost, test_y):
    le = preprocessing.LabelEncoder()
    y_t_xgboost = le.fit_transform(y_train_xgboost)
    t_y = le.fit_transform(test_y)
    return y_t_xgboost, t_y

y_tr_xgboost, test_y_xgboost = transform_labels(y_train_xgboost, test_y)

print(y_tr_xgboost)
print(test_y_xgboost)

xgboost_train_dataset = np.concatenate([np.array(x_train_xgboost).reshape(x_train.shape[0],256),np.array(y_tr_xgboost).reshape(y_tr_xgboost.shape[0],1)],axis=1)
xgboost_train_dataset = pd.DataFrame(xgboost_train_dataset)
xgboost_train_dataset.to_csv('train_xgboost.csv',index=False)

xgboost_test_dataset = np.concatenate([np.array(x_test_xgboost).reshape(x_test.shape[0],256),np.array(test_y_xgboost).reshape(test_y_xgboost.shape[0],1)],axis=1)
xgboost_test_dataset = pd.DataFrame(xgboost_test_dataset)
xgboost_test_dataset.to_csv('test_xgboost.csv',index=False)

train = pd.read_csv('train_xgboost.csv')
train_y = train['256'].astype('int')
train_x = train.drop(['256'],axis=1)

test = pd.read_csv('test_xgboost.csv')
test_y = test['256'].astype('int')
test_x = test.drop(['256'],axis=1)

dataset = xgb.DMatrix(train_x, label=train_y)
watchlist = [(dataset, 'train')]
params = {'max_depth':6, 'eta':0.1, 'silent':1, 'num_class': classes_in_dataset} 
model_xg = xgb.train(params, dataset, num_boost_round=50)


test_x = xgb.DMatrix(test_x)
result = model_xg.predict(test_x)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 470, 1)]          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 470, 64)           128       
_________________________________________________________________
activation_8 (Activation)    (None, 470, 64)           0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 469, 64)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 469, 62)           4030      
_________________________________________________________________
activation_9 (Activation)    (None, 469, 62)           0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 468, 62)           0   

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:

from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix

#preds = model.predict(d_test)
#best_preds = np.asarray([np.argmax(line) for line in result])

print("Precision = {}".format(precision_score(test_y, result, average='macro', zero_division=0)))
print("Recall = {}".format(recall_score(test_y, result, average='macro', zero_division=0)))
print("Accuracy = {}".format(accuracy_score(test_y, result)))

print(metrics.classification_report(test_y, result))
print(metrics.confusion_matrix(test_y, result))

Precision = 0.5492857142857143
Recall = 0.5333333333333333
Accuracy = 0.5333333333333333
              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.40      0.33      0.36         6
           2       0.38      0.50      0.43         6
           3       0.57      0.67      0.62         6
           4       0.60      0.50      0.55         6

    accuracy                           0.53        30
   macro avg       0.55      0.53      0.54        30
weighted avg       0.55      0.53      0.54        30

[[4 0 1 1 0]
 [1 2 1 0 2]
 [0 2 3 1 0]
 [0 0 2 4 0]
 [0 1 1 1 3]]
